In [1]:
# Install necessary packages (uncomment if running in a new environment)
# !pip3 install beautifulsoup4
# !pip3 install requests

import sys
import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

# Helper functions
def date_time(table_cells):
    """Extract date and time from the first cell."""
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """Extract booster version text from second cell."""
    out = ''.join([booster_version for i, booster_version in enumerate(table_cells.strings) if i % 2 == 0][0:-1])
    return out

def landing_status(table_cells):
    """Extract landing status from the cell."""
    out = [i for i in table_cells.strings][0]
    return out

def get_mass(table_cells):
    """Extract payload mass from the cell."""
    mass = unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass = mass[0:mass.find("kg") + 2]
    else:
        new_mass = 0
    return new_mass

def extract_column_from_header(row):
    """Clean and extract column name from table header cell."""
    if row.br:
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name

# Step 1: Load Wikipedia page
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"
response = requests.get(static_url)
soup = BeautifulSoup(response.text, "html.parser")
print("Page Title:", soup.title.string)

# Step 2: Initialize launch_dict
launch_dict = {
    'Flight No.': [],
    'Launch site': [],
    'Payload': [],
    'Payload mass': [],
    'Orbit': [],
    'Customer': [],
    'Launch outcome': [],
    'Version Booster': [],
    'Booster landing': [],
    'Date': [],
    'Time': []
}

# Step 3: Parse all relevant tables
extracted_row = 0

for table_number, table in enumerate(soup.find_all('table', class_="wikitable plainrowheaders collapsible")):
    for rows in table.find_all("tr"):
        # Check if this row starts with a numbered flight (indicating a valid launch record)
        if rows.th:
            if rows.th.string:
                flight_number = rows.th.string.strip()
                flag = flight_number.isdigit()
        else:
            flag = False

        row = rows.find_all('td')
        if flag and len(row) >= 9:
            extracted_row += 1
            
            # 1. Flight No.
            launch_dict['Flight No.'].append(flight_number)

            # 2. Date and Time
            datatimelist = date_time(row[0])
            date = datatimelist[0].strip(',')
            time = datatimelist[1]
            launch_dict['Date'].append(date)
            launch_dict['Time'].append(time)

            # 3. Version Booster
            bv = booster_version(row[1])
            if not bv and row[1].a:
                bv = row[1].a.string
            launch_dict['Version Booster'].append(bv)

            # 4. Launch Site
            launch_site = row[2].a.string if row[2].a else row[2].get_text(strip=True)
            launch_dict['Launch site'].append(launch_site)

            # 5. Payload
            payload = row[3].a.string if row[3].a else row[3].get_text(strip=True)
            launch_dict['Payload'].append(payload)

            # 6. Payload Mass
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)

            # 7. Orbit
            orbit = row[5].a.string if row[5].a else row[5].get_text(strip=True)
            launch_dict['Orbit'].append(orbit)

            # 8. Customer
            customer = row[6].a.string if row[6].a else row[6].get_text(strip=True)
            launch_dict['Customer'].append(customer)

            # 9. Launch Outcome
            launch_outcome = list(row[7].strings)[0].strip()
            launch_dict['Launch outcome'].append(launch_outcome)

            # 10. Booster Landing
            booster_landing = landing_status(row[8])
            launch_dict['Booster landing'].append(booster_landing)

# Step 4: Convert to DataFrame and export
df = pd.DataFrame({ key: pd.Series(value) for key, value in launch_dict.items() })
df.to_csv('spacex_web_scraped.csv', index=False)

print(f"\n Extracted {extracted_row} launch records.")
print("Data saved to: spacex_web_scraped.csv")


Page Title: List of Falcon 9 and Falcon Heavy launches - Wikipedia

 Extracted 121 launch records.
Data saved to: spacex_web_scraped.csv
